## Import libraries and load data


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("course_lead_scoring.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


## Data Preparation

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [4]:
df.dtypes[df.dtypes == 'object'].index

Index(['lead_source', 'industry', 'employment_status', 'location'], dtype='object')

In [5]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']

In [6]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

In [7]:
df[categorical] = df[categorical].fillna('NA')
df[categorical].isnull().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [8]:
df[numerical] = df[numerical].fillna(0.0)
df[numerical].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
dtype: int64

## Question 1

What is the most frequent observation (mode) for the column industry?

In [9]:
df.industry.value_counts(ascending=False).index[0]

'retail'

## Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

In [10]:
correlation_matrix = df[numerical].corr()
correlation_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [11]:
#Splitting the data

from sklearn.model_selection import train_test_split 

In [12]:
df_train_full, df_test = train_test_split(df, test_size = 0.2, random_state = 42)

In [13]:
df_train, df_val  = train_test_split(df_train_full, test_size = 0.25, random_state = 42)

In [14]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [15]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [16]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [17]:
del df_train['converted']
del df_val['converted']
del df_test['converted'] 

In [18]:
df_train_full

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
1066,social_media,manufacturing,2,44403.0,self_employed,australia,1,0.71,0
638,events,retail,3,38048.0,student,north_america,6,0.97,1
799,social_media,education,2,71399.0,NA,europe,1,0.51,1
380,referral,education,2,47912.0,employed,australia,1,0.04,0
303,paid_ads,healthcare,1,34806.0,employed,europe,4,0.32,1
...,...,...,...,...,...,...,...,...,...
1130,events,manufacturing,1,57039.0,employed,south_america,2,0.30,0
1294,events,healthcare,2,56185.0,student,south_america,2,0.44,0
860,paid_ads,manufacturing,1,56402.0,student,north_america,1,0.02,0
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1


## Question 3

* Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.

* Round the scores to 2 decimals using round(score, 2).

In [19]:
from sklearn.metrics import mutual_info_score

In [24]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, df_train_full.converted)

In [25]:
mi = df_train_full[categorical].apply(mutual_info_converted_score)
round(mi.sort_values(ascending=False), 2)

lead_source          0.03
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

## Question 4

* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
 To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
 model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [26]:
 from sklearn.feature_extraction import DictVectorizer

In [27]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [28]:
dv = DictVectorizer(sparse=False)

In [29]:
X_train = dv.fit_transform(train_dicts)

In [30]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

In [31]:
X_val = dv.transform(val_dicts)

In [32]:
#Fitting the LR model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [33]:
model.intercept_[0]

-0.0691472802783413

In [34]:
model.coef_[0]

array([-1.77843866e-05, -1.47154423e-02,  3.39095225e-02,  2.66248432e-03,
        1.15238518e-02, -1.02527697e-01, -2.48510995e-02,  4.93604222e-02,
       -2.01258344e-02, -1.34214865e-02, -3.00232200e-03, -9.25991830e-03,
       -3.17957304e-02, -1.60513114e-02,  3.11339155e-01,  5.12012528e-02,
        2.01511698e-02, -1.20346284e-02, -1.16021521e-02, -1.15251880e-01,
        7.95303436e-02, -2.99401329e-02,  3.95843295e-03, -1.14296944e-02,
       -1.12457415e-02, -5.59987025e-03,  8.26402635e-03,  5.58598769e-03,
       -3.33967159e-02, -2.52837052e-02,  4.53752887e-01])

In [35]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.61192163, 0.79982617, 0.53021344, 0.47131479, 0.57066132,
       0.44227169, 0.87127669, 0.84883115, 0.83290038, 0.61497802,
       0.54968027, 0.78153088, 0.69039786, 0.77017122, 0.52659441,
       0.91706425, 0.53170635, 0.42123049, 0.30146455, 0.84881583,
       0.79488653, 0.73670375, 0.44527211, 0.64838383, 0.4176882 ,
       0.75393418, 0.90166116, 0.33903049, 0.43181431, 0.9680681 ,
       0.92018714, 0.37487989, 0.652301  , 0.90650057, 0.75164117,
       0.64202121, 0.82250075, 0.83375553, 0.659116  , 0.30978854,
       0.78942264, 0.35546366, 0.96517758, 0.63389304, 0.51274195,
       0.53230534, 0.82287785, 0.744074  , 0.73452314, 0.68955217,
       0.46964443, 0.84539252, 0.55635244, 0.92637871, 0.65258021,
       0.61526273, 0.63816996, 0.28304018, 0.48049824, 0.57890618,
       0.35497342, 0.62175051, 0.38960778, 0.61156056, 0.85304278,
       0.75430136, 0.89185954, 0.71946459, 0.95387623, 0.89209517,
       0.75277088, 0.33850139, 0.61376593, 0.51622275, 0.64088

In [36]:
#Prediction decision

converted_decision = (y_pred >= 0.5)

In [37]:
#model accuracy

round((converted_decision ==y_val).mean(), 2)

0.7

## Question 5

Let's find the least useful feature using the feature elimination technique.

Train a model using the same features and parameters as in Q4 (without rounding).

Now exclude each feature from this set and train a model without it. Record the accuracy for each model.

For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

In [38]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Baseline accuracy

y_pred = model.predict_proba(X_train)[:, 1]
converted_decision = (y_pred >= 0.5)
baseline_accuracy = (converted_decision ==y_train).mean()
print("Baseline Accuracy:", baseline_accuracy)


Baseline Accuracy: 0.7385844748858448


In [39]:
features = numerical + categorical
features

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'lead_source',
 'industry',
 'employment_status',
 'location']

In [40]:
from IPython.display import display

In [41]:
for feature in features:
    reduced_features = [f for f in features if f != feature]
    
    train_dicts = df_train[reduced_features].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(train_dicts)
    #val_dicts = df_val[reduced_feature].to_dict(orient='records')
    #X_val = dv.transform(val_dicts)
    
    
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_train)[:, 1]
    
    converted_decision = (y_pred >= 0.5)
    accuracy = (converted_decision ==y_train).mean()
    diff = baseline_accuracy - accuracy
    #display(reduced_features)
    display(feature, diff)
    print()
    print()

'number_of_courses_viewed'

0.11529680365296802

'annual_income'

-0.13242009132420085

'interaction_count'

...

-0.0022831050228310223

'employment_status'

0.003424657534246589

'location'

-0.003424657534246589

## Question 6

* Now let's train a regularized logistic regression.
* Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
* Train models using all the features as in Q4.
* Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

In [44]:

    
train_dicts = df_train[numerical + categorical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[numerical + categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
    
C_values = [0.01, 0.1, 1, 10, 100]

for c in C_values:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    
    converted_decision = (y_pred >= 0.5)
    accuracy = round((converted_decision ==y_val).mean(),2)

    display(c, accuracy)
    print()
    print()

0.01

0.7

0.1

0.7

1

0.7

10

0.7

100

0.7

In [ ]:
df_train